In [0]:
!pip install -U -q kaggle
!mkdir -p ~/.kaggle
from google.colab import files
import os

In [0]:
%tensorflow_version 1.x
import tensorflow
import os
import tqdm
import matplotlib.pyplot as plt
from keras import preprocessing, layers, models, optimizers
import numpy as np

In [0]:
files.upload()

In [0]:
image_size = 299
batch_size = 32

In [0]:
!cp kaggle.json ~/.kaggle/

In [0]:
!kaggle datasets download -d paultimothymooney/chest-xray-pneumonia

In [0]:
!chmod 600 /root/.kaggle/kaggle.json

In [0]:
!ls chest_xray/

In [0]:
!unzip chest-xray-pneumonia.zip

In [0]:
train_dir = 'chest_xray/train'
test_dir = 'chest_xray/test'
input_shape = (image_size,image_size,3)


In [0]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu',
                       input_shape=input_shape))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dropout(0.5))                       # DROPOUT
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',
             optimizer=optimizers.RMSprop(lr=1e-4),
             metrics=['acc'])
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '4'

In [0]:
model.summary()

In [0]:
train_datagen = preprocessing.image.ImageDataGenerator(rescale=1. / 255,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    fill_mode='nearest')

In [0]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(image_size, image_size),
    batch_size=batch_size,
    class_mode='binary')

In [0]:
test_datagen = preprocessing.image.ImageDataGenerator(rescale=1. / 255)

In [0]:
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(image_size, image_size),
    batch_size=batch_size,
    class_mode='binary')

In [0]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch=40,
    epochs= 20, # use more epochs if you are not limited by 1 hour limit
    validation_data=test_generator,
    validation_steps=5)

In [0]:
model.summary()

In [0]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.ylim(0,2)
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# График оценки loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.ylim(0, 2)
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [0]:
from sklearn.metrics import classification_report, confusion_matrix

In [0]:
import numpy as np

In [0]:
val_dir = 'chest_xray/val'
val_datagen = preprocessing.image.ImageDataGenerator(rescale=1. / 255)
val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(image_size, image_size),
    batch_size=batch_size,
    class_mode='binary')

In [0]:
true_res = list()
predicted_res = list()
for i in val_generator.filenames:
    path = i.split("/")
    if (path[0] == "NORMAL"):
      true_res.append(0)
    else:
      true_res.append(1)
    path_im = "chest_xray/val/" + i
    img = preprocessing.image.load_img(path_im, target_size=(image_size, image_size))
    plt.imshow(img)
    plt.title(path[0])
    plt.show()  
    x = preprocessing.image.img_to_array(img)
    x /=255
    x = np.expand_dims(x, axis=0)
    prediction = model.predict(x)
    print(prediction)
    if (prediction <=0.5):
      predicted_res.append(0)
    else:
      predicted_res.append(1)

In [0]:
import itertools

In [0]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [0]:
font = {'size' : 15}

plt.rc('font', **font)

cnf_matrix = confusion_matrix(true_res,predicted_res)
plt.figure(figsize=(10, 8))
plot_confusion_matrix(cnf_matrix, classes=["NORMAL", "PNEUMONIA"],
                      title='Confusion matrix')
plt.savefig("conf_matrix.png")
plt.show()